<a href="https://colab.research.google.com/github/crakcode/MLFintech/blob/main/Lecture_Crawler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 리퀘스트와 뷰티풀수프를 이용한 크롤링

### requests.get을 이용한 웹 페이지 소스 수집

In [ ]:
import requests
url = "https://news.daum.net/ranking/popular?regDate=20211015"
agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.45 Safari/537.36"
response = requests.get(url = url, headers = {"User-Agent":agent})

print("hello world")

#### response 객체

In [ ]:
print(type(response))

<class 'requests.models.Response'>


In [ ]:
print(response.status_code)

200


In [ ]:
res_text = response.text
res_content = response.content

print(res_text[:200])
print('=' * 200)
print(res_content[:200])


<!DOCTYPE html>



<html lang="ko" class="os_windows chrome pc version_96_0_4664_45 ">
<head>
<meta charset="utf-8">
<meta name="referrer" content="always" />

<meta property="og:author" content="Dau
b'\n<!DOCTYPE html>\n\n\n\n<html lang="ko" class="os_windows chrome pc version_96_0_4664_45 ">\n<head>\n<meta charset="utf-8">\n<meta name="referrer" content="always" />\n\n<meta property="og:author" content="Dau'


In [ ]:
print(type(res_text)) # <class "str">
print(type(res_content)) # <class "bytes">

<class 'str'>
<class 'bytes'>


### BeautifulSoup을 이용한 파싱

In [ ]:
from bs4 import BeautifulSoup as BS

In [ ]:
parsed_res_text = BS(res_text)
print(type(parsed_res_text))

<class 'bs4.BeautifulSoup'>


#### find와 find_all 메서드

In [ ]:
# 첫 번째 ul 태그 찾기
print(parsed_res_text.find("ul"))

<ul class="doc-relate" data-tiara-layer="GNB service">
<li><a class="link_services" data-tiara-layer="enter" href="https://entertain.daum.net">연예</a></li>
<li><a class="link_services" data-tiara-layer="sports" href="https://sports.daum.net">스포츠</a></li>
</ul>


In [ ]:
# 세 번째 위치의 ul 태그 찾기
print(parsed_res_text.find_all("ul")[3])

In [ ]:
# class 속성이 info_news인 첫 번째 span 태그 찾기
# print(parsed_res_text.find("span", attrs = {"class":"info_news"}))
print(parsed_res_text.find("span", attrs = {"class":"txt_info"}))

<span class="txt_info">YTN</span>


In [ ]:
# 내용이 조선 일보인 모든 span 태그 찾기
print(parsed_res_text.find_all("span", text = "서울신문"))

[]


In [ ]:
#ul_tag = parsed_res_text.find("ul", attrs = {"class":"list_news2"})
ul_tag = parsed_res_text.find("div", attrs = {"class":"box_g box_news_issue"})
print(type(ul_tag))

<class 'bs4.element.Tag'>


In [ ]:
li_list = ul_tag.find_all("li")

In [ ]:
li_list[0]

<li>
<div class="item_issue" data-tiara-layer="headline1">
<a class="wrap_thumb" data-tiara-custom="contentUniqueKey=hamny-20220603230851095" data-tiara-id="20220603230851095" data-tiara-layer="article_main" data-tiara-ordnum="1" data-tiara-type="harmony" href="https://v.daum.net/v/20220603230851095">
<img alt="" class="thumb_g" src="https://img1.daumcdn.net/thumb/S96x60ht.u/?fname=https%3A%2F%2Ft1.daumcdn.net%2Fnews%2F202206%2F03%2Fyonhap%2F20220603230851302nugd.jpg&amp;scode=media"/>
</a>
<div class="cont_thumb">
<span class="info_thumb">
<span class="logo_cp">
<img alt="연합뉴스" class="thumb_g" src="https://t1.daumcdn.net/media/news/news2016/cp/cp_yonhap.gif"/>
</span>
<span class="txt_category">국제</span>
</span>
<strong class="tit_g">
<a class="link_txt" data-tiara-custom="contentUniqueKey=hamny-20220603230851095" data-tiara-id="20220603230851095" data-tiara-layer="article_main" data-tiara-ordnum="1" data-tiara-type="harmony" href="https://v.daum.net/v/20220603230851095">
            

In [ ]:
news_name = li_list[0].find("a", attrs = {"class":"link_txt"})
print(news_name) 

<a class="link_txt" data-tiara-custom="contentUniqueKey=hamny-20220603230851095" data-tiara-id="20220603230851095" data-tiara-layer="article_main" data-tiara-ordnum="1" data-tiara-type="harmony" href="https://v.daum.net/v/20220603230851095">
                                    개전 100일 맞은 젤렌스키 "승리는 우리의 것"[우크라 침공]
                                </a>


In [ ]:
# print(parsed_res_text.find("span", attrs = {"class":"txt_info"}))

In [ ]:
li_list[0].select('img')[1]['alt']

'연합뉴스'

In [ ]:
#press_name = li_list[0].find("span", attrs = {"class":"info_news"})
press_name = li_list[0].select('img')[1]['alt']
print(press_name) 

연합뉴스


#### 속성을 이용한 값 가져오기

In [ ]:
print(news_name["href"])

https://v.daum.net/v/20220603230851095


In [ ]:
print(news_name.text)


                                    개전 100일 맞은 젤렌스키 "승리는 우리의 것"[우크라 침공]
                                


In [ ]:
print(press_name)

연합뉴스


In [ ]:
import pandas as pd 
# 빈 리스트로 초기화
news_name_list = []
news_link_list = []
press_name_list = []
for li in li_list: # li를 li_list를 순회하면서
    try:
        # class가 link_txt인 a 태그를 news_name_tag에 저장
        news_name_tag = li.find("a", attrs = {"class":"link_txt"})
        #  press_name_tag = li.find("span", attrs = {"class":"txt_info"})

        news_name = news_name_tag.text.replace("\n", "")
        press_name = li.select('img')[1]['alt']
        news_link = news_name_tag["href"]

        news_name_list.append(news_name)
        press_name_list.append(press_name)
        news_link_list.append(news_link)
    except:
        pass

data = pd.DataFrame({"뉴스명":news_name_list, 
                     "언론사명":press_name_list,
                    "뉴스링크":news_link_list})

display(data.head(10)) 

,뉴스명,언론사명,뉴스링크
0,개전 100일 맞은...,연합뉴스,https://v.daum.net/v/20220603230851095
1,中이 반도체 공장 ...,뉴시스,https://v.daum.net/v/20220603230740085
2,"美국무부 부장관, ...",연합뉴스,https://v.daum.net/v/20220603230142038
3,"민주 ""文 사저 앞...",연합뉴스,https://v.daum.net/v/20220603225925025
4,"민주당 내전 ""당권...",YTN,https://v.daum.net/v/20220603225820017
5,아마존 킨들·링크트...,조선일보,https://v.daum.net/v/20220603225610997
6,미 일자리 39만개...,이데일리,https://v.daum.net/v/20220603224532875
7,"국힘, 혁신위서 공...",연합뉴스,https://v.daum.net/v/20220603224118851
8,"정부, 가뭄 확산에...",뉴시스,https://v.daum.net/v/20220603223835815
9,내 폰은 왜 고속 ...,IT동아,https://v.daum.net/v/20220603223001720


# 증권사 종목 리포트 크롤링하기(프로젝트)


## 구글 드라이브 연경

In [ ]:
import os 
from google.colab import drive
drive.mount('/content/drive')
os.chdir("/content/drive/MyDrive/금융특화 계층별 AI역량강화 교안/실습1_5. 웹 크롤링과 금융 데이터/")

Mounted at /content/drive


#### 파이썬을 이용한 리포트 수집

In [ ]:
import requests
from tqdm import tqdm
from bs4 import BeautifulSoup as BS

In [ ]:
url = "http://www.paxnet.co.kr/stock/report/report?wlog_rpt=jm&menuCode=2222"
response = requests.get(url)
html = response.content
html = BS(html)

In [ ]:
div = html.find("div", attrs = {"class":"board-type"})
li_list = div.find_all("li")

#### 수집 대상 행 결정

In [ ]:
print(li_list[0])

<li class="board-list-th">
<div>종목</div>
<div>제목</div>
<div class="right">적정가격</div>
<div>투자의견</div>
<div>제공출처</div>
<div class="cent">작성일</div>
</li>


In [ ]:
print(li_list[1])

<li>
<div><strong class="color-cate"><span class="bracket">[</span><a href="http://www.paxnet.co.kr/stock/analysis/main?wlog_rpt=list_jm&amp;abbrSymbol=045390">대아티아이</a><span class="bracket">]</span></strong></div>
<div>
<p>
<a href="javascript:selectView('137005');">철도 산업의 미래는 대아티아이가 중심 </a>
</p>
</div>
<div class="line3">
<span>적정가격</span>
	 								
										
										
											-원
										
									
								</div>
<div class="line3 color-black">
									-
								</div>
<div class="line3">SK증권</div>
<div class="line3">2022.06.03</div>
</li>


In [ ]:
print(li_list[-1])

<li><a href="#" onclick="linkPage(5); return false;">5</a></li>


In [ ]:
check_str = '<strong class="color-cate"><span class="bracket">'

In [ ]:
print(check_str in str(li_list[1]))
print(check_str in str(li_list[0]))
print(check_str in str(li_list[-1]))

True
False
False


#### 파싱 전략 수립

In [ ]:
stock = li_list[1].find_all("a")[0].text
print(stock)

대아티아이


In [ ]:
title = li_list[1].find_all("a")[1].text
print(title)

철도 산업의 미래는 대아티아이가 중심 


In [ ]:
price = li_list[1].find("div", attrs = {"class":"line3"}).text
display(price)

'\n적정가격\r\n\t \t\t\t\t\t\t\t\t\r\n\t\t\t\t\t\t\t\t\t\t\r\n\t\t\t\t\t\t\t\t\t\t\r\n\t\t\t\t\t\t\t\t\t\t\t-원\r\n\t\t\t\t\t\t\t\t\t\t\r\n\t\t\t\t\t\t\t\t\t\r\n\t\t\t\t\t\t\t\t'

In [ ]:
price = price.replace('\r', '').replace('\t', '').replace('\n', '')
price = price.split(' ')[1].replace(',', '')[:-1]
display(price)

'-'

In [ ]:
opinion = li_list[1].find_all("div", attrs = {"class":"line3"})[1].text
display(opinion)

'\r\n\t\t\t\t\t\t\t\t\t-\r\n\t\t\t\t\t\t\t\t'

In [ ]:
opinion = opinion.replace('\t', '').replace('\n', '').replace('\r', '')
display(opinion)

'-'

In [ ]:
trading_firm = li_list[1].find_all("div", attrs = {"class":"line3"})[2].text
display(trading_firm)

'SK증권'

In [ ]:
date = li_list[1].find_all("div", attrs = {"class":"line3"})[3].text
display(date)

'2022.06.03'

In [ ]:
import pandas as pd
date = pd.to_datetime(date)
display(date)

Timestamp('2022-06-03 00:00:00')

#### 단일 리포트 파싱

In [ ]:
def parsing_li(li):
    stock = li.find_all("a")[0].text # 종목명
    title = li.find_all("a")[1].text # 리포트 제목
    price = li.find("div", attrs = {"class":"line3"}).text # 적정가격
    price = price.replace('\r', '').replace('\t', '').replace('\n', '')
    price = price.split(' ')[1].replace(',', '')[:-1]
    opinion = li.find_all("div", attrs = {"class":"line3"})[1].text # 의견
    opinion = opinion.replace('\t', '').replace('\n', '').replace('\r', '')
    trading_firm = li.find_all("div", attrs = {"class":"line3"})[2].text # 증권사
    date = li.find_all("div", attrs = {"class":"line3"})[3].text # 날짜
    date = pd.to_datetime(date)
    
    return [stock, title, price, opinion, trading_firm, date]

In [ ]:
print(parsing_li(li_list[1]))

['대아티아이', '철도 산업의 미래는 대아티아이가 중심 ', '-', '-', 'SK증권', Timestamp('2022-06-03 00:00:00')]


#### 한 페이지 내 모든 리포트 수집 및 정제

In [ ]:
data = []
for li in li_list:
    if check_str in str(li):
        record = parsing_li(li)
        data.append(record)
data = pd.DataFrame(data, columns = ["종목명", "리포트제목", "적정가격", "의견", "증권사", "날짜"])

In [ ]:
display(data.head(10))

,종목명,리포트제목,적정가격,의견,증권사,날짜
0,대아티아이,철도 산업의 미래는 대아티아이가 중심,-,-,SK증권,2022-06-03
1,에스티아이,"거래선, 아이템 다변화 동시 진행 중",-,-,SK증권,2022-06-03
2,형지I&C,반등의 지렛대가 될 2022 년,-,-,SK증권,2022-06-03
3,한세실업,1Q22 Review: 숫자로 증명한다,45000,매수,SK증권,2022-06-03
4,엘앤에프,프리미엄의 근거(Feat. 고객사 낙수효과),430000,매수,메리츠증권,2022-06-03
5,코스맥스,1Q22 Review: 위기 속에 드러난 민낯,70000,Hold,메리츠증권,2022-06-03
6,엔씨소프트,脫(탈) 리니지 4Q22 부터 추진,550000,매수,SK증권,2022-06-03
7,대한해운,중국 봉쇄 긍정적이지만 컨테이너선 부재가 아쉽다,3500,매수(유지),SK증권,2022-06-03
8,SK,호실적이 주가상승에 대한 확신으로,365000,매수(유지),SK증권,2022-06-03
9,SFA반도체,Bump 사업 이익기여 예상 상회,9200,매수(유지),SK증권,2022-06-03


In [ ]:
import time
base_url = "http://www.paxnet.co.kr/stock/report/report?menuCode=2222&currentPageNo={}&reportId=0&searchKey=stock&searchValue="
data = []

# for page_no in tqdm(range(1, 563)):
# 가볍게 30개만 해보기
for page_no in tqdm(range(1, 31)):
    # 웹 페이지 소스 가져오기
    url = base_url.format(page_no)
    while True:
        response = requests.get(url)
        if response.status_code == 200:
            time.sleep(1)
            break
        else:
            time.sleep(10 * 60)    
    html = response.content
    html = BS(html)
    
    # 웹 페이지 소스에서 필요한 부분만 가져오기
    div = html.find("div", attrs = {"class":"board-type"})
    li_list = div.find_all("li")
    for li in li_list:
        if check_str in str(li):
            record = parsing_li(li)
            data.append(record)

data = pd.DataFrame(data, columns = ["종목명", "리포트제목", "적정가격", "의견", "증권사", "날짜"])    

100%|██████████| 30/30 [00:59<00:00,  1.97s/it]


In [ ]:
data.to_csv("데이터/종목리포트.txt", sep = "\t", encoding = "euc-kr", index = False)

In [ ]:
data

,종목명,리포트제목,적정가격,의견,증권사,날짜
0,대아티아이,철도 산업의 미래는 대아티아이가 중심,-,-,SK증권,2022-06-03
1,에스티아이,"거래선, 아이템 다변화 동시 진행 중",-,-,SK증권,2022-06-03
2,형지I&C,반등의 지렛대가 될 2022 년,-,-,SK증권,2022-06-03
3,한세실업,1Q22 Review: 숫자로 증명한다,45000,매수,SK증권,2022-06-03
4,엘앤에프,프리미엄의 근거(Feat. 고객사 낙수효과),430000,매수,메리츠증권,2022-06-03
...,...,...,...,...,...,...
895,스튜디오드래곤,2022년 최대 방영편수에 도전,130000,매수(유지),키움증권,2022-02-11
896,KT&G,6% 배당수익률과 3가지 관전 포인트,103000,매수(유지),키움증권,2022-02-11
897,금호석유,경쟁사 대비 차별화된 마진율 기록,280000,매수(유지),키움증권,2022-02-11
898,하나금융지주,"4분기, 순이자마진 증가로 업계 최대 실적 달성",68000,매수(유지),키움증권,2022-02-11


# 셀레니움을 이용한 크롤링

### 모듈 및 드라이버 설치

In [ ]:
!pip install selenium
!apt-get update
!apt install chromium-chromedriver

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 983 kB 5.3 MB/s 
     |████████████████████████████████| 359 kB 47.3 MB/s 
     |████████████████████████████████| 138 kB 49.8 MB/s 
     |████████████████████████████████| 55 kB 1.6 MB/s 
     |████████████████████████████████| 4.0 MB 41.2 MB/s 
     |████████████████████████████████| 58 kB 2.7 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.9 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
Hit:1 https://d

In [ ]:
from bs4 import BeautifulSoup
import requests
import re
import time
import pandas as pd 
from tqdm import tqdm 
from selenium import webdriver
from selenium.webdriver.common.by import By

/usr/local/lib/python3.7/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


### 셀레니움 기초

#### 드라이버 연결

In [ ]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver', chrome_options=chrome_options)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: use options instead of chrome_options
  """


In [ ]:
# 페이지 url 형식에 맞게 바꾸어 주는 함수 만들기
  #입력된 수를 1, 11, 21, 31 ...만들어 주는 함수
def makePgNum(num):
    if num == 1:
        return num
    elif num == 0:
        return num+1
    else:
        return num+9*(num-1)


# 크롤링할 url 생성하는 함수 만들기(검색어, 크롤링 시작 페이지, 크롤링 종료 페이지)
def makeUrl(search,start_pg,end_pg):
    if start_pg == end_pg:
        start_page = makePgNum(start_pg)
        url = "https://search.naver.com/search.naver?where=news&sm=tab_pge&query=" + search + "&start=" + str(start_page)
        print("생성url: ",url)
        return url
    else:
        urls= []
        for i in range(start_pg,end_pg+1):
            page = makePgNum(i)
            url = "https://search.naver.com/search.naver?where=news&sm=tab_pge&query=" + search + "&start=" + str(page)
            urls.append(url)
        print("생성url: ",urls)
        return urls

In [ ]:
#검색어 입력
search = input("검색할 키워드를 입력해주세요:")

#검색 시작할 페이지 입력
page = int(input("\n크롤링할 시작 페이지를 입력해주세요. ex)1(숫자만입력):")) # ex)1 =1페이지,2=2페이지...
print("\n크롤링할 시작 페이지: ",page,"페이지")   
#검색 종료할 페이지 입력
page2 = int(input("\n크롤링할 종료 페이지를 입력해주세요. ex)1(숫자만입력):")) # ex)1 =1페이지,2=2페이지...
print("\n크롤링할 종료 페이지: ",page2,"페이지")   

# naver url 생성
search_urls = makeUrl(search,page,page2)

검색할 키워드를 입력해주세요:삼성전자

크롤링할 시작 페이지를 입력해주세요. ex)1(숫자만입력):1

크롤링할 시작 페이지:  1 페이지

크롤링할 종료 페이지를 입력해주세요. ex)1(숫자만입력):3

크롤링할 종료 페이지:  3 페이지
생성url:  ['https://search.naver.com/search.naver?where=news&sm=tab_pge&query=삼성전자&start=1', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=삼성전자&start=11', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=삼성전자&start=21']


### Page가 얼마안돼도 시간이좀 걸립니다.




In [ ]:
driver.implicitly_wait(3)


# selenium으로 검색 페이지 불러오기 #

naver_urls=[]

for i in tqdm(search_urls):
    driver.get(i)
    time.sleep(1) #대기시간 변경 가능

    # 네이버 기사 눌러서 제목 및 본문 가져오기#
    # 네이버 기사가 있는 기사 css selector 모아오기
    a = driver.find_elements(By.CSS_SELECTOR,'a.info')

    # 위에서 생성한 css selector list 하나씩 클릭하여 본문 url얻기
    for i in a:
        i.click()

        # 현재탭에 접근
        driver.switch_to.window(driver.window_handles[1])
        time.sleep(3) #대기시간 변경 가능

        # 네이버 뉴스 url만 가져오기

        url = driver.current_url
        print(url)

        if "news.naver.com" in url:
            naver_urls.append(url)
  
        else:
            pass
        
        # 현재 탭 닫기
        driver.close()

        # 다시처음 탭으로 돌아가기(매우 중요!!!)
        driver.switch_to.window(driver.window_handles[0])

print(naver_urls)

  0%|          | 0/3 [00:00<?, ?it/s]

https://www.seoul.co.kr/
https://n.news.naver.com/mnews/article/081/0003277782?sid=101
https://newsis.com/
https://n.news.naver.com/mnews/article/003/0011225273?sid=101
https://www.edaily.co.kr/
https://www.edaily.co.kr/popup/viewpopup.html?v=5
https://n.news.naver.com/mnews/article/018/0005233147?sid=101
https://www.news1.kr/
https://n.news.naver.com/mnews/article/421/0006135487?sid=101
https://www.fnnews.com/
https://n.news.naver.com/mnews/article/014/0004846235?sid=101
https://www.hankyung.com/
https://n.news.naver.com/mnews/article/015/0004706883?sid=101
https://static.hankyung.com/popup/220520/
https://www.nocutnews.co.kr/
https://n.news.naver.com/mnews/article/079/0003650451?sid=101
https://www.chosun.com/
https://n.news.naver.com/mnews/article/023/0003695467?sid=101
https://www.sedaily.com/


 33%|███▎      | 1/3 [02:14<04:28, 134.31s/it]

https://n.news.naver.com/mnews/article/011/0004061246?sid=101
https://moneys.mt.co.kr/
https://n.news.naver.com/mnews/article/417/0000819467?sid=105
https://newsis.com/
https://n.news.naver.com/mnews/article/003/0011228255?sid=101
https://www.hankyung.com/
https://static.hankyung.com/popup/220520/
https://n.news.naver.com/mnews/article/015/0004707198?sid=105
https://www.yna.co.kr/
https://n.news.naver.com/mnews/article/001/0013224773?sid=101
https://www.yna.co.kr/
https://n.news.naver.com/mnews/article/001/0013222336?sid=101
https://www.yna.co.kr/
https://n.news.naver.com/mnews/article/001/0013221725?sid=101
http://www.bizwatch.co.kr/
https://n.news.naver.com/mnews/article/648/0000008323?sid=101
https://newsis.com/
https://n.news.naver.com/mnews/article/003/0011225209?sid=101
https://www.yna.co.kr/
https://n.news.naver.com/mnews/article/001/0013221438?sid=101


 67%|██████▋   | 2/3 [03:33<01:42, 102.12s/it]

https://www.edaily.co.kr/
https://n.news.naver.com/mnews/article/018/0005232591?sid=101
https://www.edaily.co.kr/popup/viewpopup.html?v=3
https://zdnet.co.kr/
https://n.news.naver.com/mnews/article/092/0002258703?sid=105
http://biz.heraldcorp.com/
https://n.news.naver.com/mnews/article/016/0001999102?sid=105
https://www.chosun.com/
https://n.news.naver.com/mnews/article/023/0003695331?sid=105
https://biz.sbs.co.kr/
https://n.news.naver.com/mnews/article/374/0000288654?sid=101
https://zdnet.co.kr/
https://n.news.naver.com/mnews/article/092/0002258579?sid=105
https://www.edaily.co.kr/
https://n.news.naver.com/mnews/article/018/0005232175?sid=101
https://www.edaily.co.kr/popup/viewpopup.html?v=6
https://newsis.com/
https://n.news.naver.com/mnews/article/003/0011224609?sid=101
https://moneys.mt.co.kr/
https://n.news.naver.com/mnews/article/417/0000819138?sid=101


100%|██████████| 3/3 [04:47<00:00, 95.74s/it]

https://magazine.hankyung.com/
['https://n.news.naver.com/mnews/article/081/0003277782?sid=101', 'https://n.news.naver.com/mnews/article/003/0011225273?sid=101', 'https://n.news.naver.com/mnews/article/018/0005233147?sid=101', 'https://n.news.naver.com/mnews/article/421/0006135487?sid=101', 'https://n.news.naver.com/mnews/article/014/0004846235?sid=101', 'https://n.news.naver.com/mnews/article/015/0004706883?sid=101', 'https://n.news.naver.com/mnews/article/079/0003650451?sid=101', 'https://n.news.naver.com/mnews/article/023/0003695467?sid=101', 'https://n.news.naver.com/mnews/article/011/0004061246?sid=101', 'https://n.news.naver.com/mnews/article/417/0000819467?sid=105', 'https://n.news.naver.com/mnews/article/003/0011228255?sid=101', 'https://n.news.naver.com/mnews/article/015/0004707198?sid=105', 'https://n.news.naver.com/mnews/article/001/0013224773?sid=101', 'https://n.news.naver.com/mnews/article/001/0013222336?sid=101', 'https://n.news.naver.com/mnews/article/001/0013221725?sid

In [ ]:
naver_urls

['https://n.news.naver.com/mnews/article/081/0003277782?sid=101',
 'https://n.news.naver.com/mnews/article/003/0011225273?sid=101',
 'https://n.news.naver.com/mnews/article/018/0005233147?sid=101',
 'https://n.news.naver.com/mnews/article/421/0006135487?sid=101',
 'https://n.news.naver.com/mnews/article/014/0004846235?sid=101',
 'https://n.news.naver.com/mnews/article/015/0004706883?sid=101',
 'https://n.news.naver.com/mnews/article/079/0003650451?sid=101',
 'https://n.news.naver.com/mnews/article/023/0003695467?sid=101',
 'https://n.news.naver.com/mnews/article/011/0004061246?sid=101',
 'https://n.news.naver.com/mnews/article/417/0000819467?sid=105',
 'https://n.news.naver.com/mnews/article/003/0011228255?sid=101',
 'https://n.news.naver.com/mnews/article/015/0004707198?sid=105',
 'https://n.news.naver.com/mnews/article/001/0013224773?sid=101',
 'https://n.news.naver.com/mnews/article/001/0013222336?sid=101',
 'https://n.news.naver.com/mnews/article/001/0013221725?sid=101',
 'https://

In [ ]:

# ConnectionError방지
headers = { "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) Chrome/98.0.4758.102" }

titles = []
contents=[]
for i in naver_urls:
    original_html = requests.get(i,headers=headers)
    html = BeautifulSoup(original_html.text, "html.parser")
    # 검색결과확인시
    # print(html)
    
    #뉴스 제목 가져오기
    title = html.select("#ct > div.media_end_head.go_trans > div.media_end_head_title")
    # list합치기
    title = ''.join(str(title))
    # html태그제거
    pattern1 = '<[^>]*>'
    title = re.sub(pattern=pattern1,repl='',string=title)
    titles.append(title)

    #뉴스 본문 가져오기

    content = html.select("#dic_area")

    # 기사 텍스트만 가져오기
    # list합치기
    # print(content)
    content = ''.join(str(content))
    
    #html태그제거 및 텍스트 다듬기
    content = re.sub(pattern=pattern1,repl='',string=content)
    pattern2 = """[\n\n\n\n\n// flash 오류를 우회하기 위한 함수 추가\nfunction _flash_removeCallback() {}"""
    content = content.replace(pattern2,'')

    contents.append(content)

print(titles)
print(contents)

['[\n이찬희 삼성 준법위원장 “이재용 사면 결단내려야”\n]', '[\n이재승 삼성전자 사장 "국내 판매 냉장고 3대 중 2대 비스포크"\n]', '[\n이찬희 삼성 준법위원장 “이재용 부회장 사면, 국민의 뜻”\n]', "[\n대기업 고용인원 170만명 1년새 6만명↑…삼성전자 11만명 '최대'\n]", '[\n삼성전자, 반도체연구소장 등 임원인사 단행\n]', '[\n삼성전자에 무슨 일이…반도체 임원 20여명 교체\n]', "[\n삼성전자, 브라질서 에어컨 전력량 관리 '스마트싱스 에너지' 출시\n]", '[\n삼성스팩 이틀 연속 상한가, 증권가도 당황했다… 무슨 일?\n]', "[\n반도체 부사장 10명 교체…급박하게 돌아가는 '삼성시계' [뒷북비즈]\n]", "[\n삼성전자, 1분기 스마트워치 시장 2위… '갤럭시워치4'가 점유율 늘려\n]", '[\n삼성전자, 임원까지 주식 매입 나섰지만…주가 한 달째 제자리\n]', "[\n삼성, 폴더블폰 독주…'글로벌 점유율 74%' 압도적 1위\n]", "[\n삼성전자, 브라질서 전력량 관리 '스마트싱스 에너지' 출시\n]", "[\n삼성전자, '페이퍼 프리' 시스템 전세계 서비스센터로 확대 적용\n]", '[\n삼성전자, 비스포크 냉장고 출시 3주년 기념 한정판 패널 판매\n]', '[\n삼성전자, 반도체 전열 다시 정비했다\n]', "[\n삼성전자, 180개국 글로벌 서비스센터서 '전자영수증' 발급\n]", '[\n20대 기업 현금성 자산 226조원…삼성전자 124조원\n]', "[\n'6만전자' 묶인 삼성전자…M&amp;A로 분위기 반전할까\n]", '[\n삼성 스마트싱스가 서버리스와 플링크 택한 이유\n]', '[\n“이게 겨우 10만원, 진짜야?” 삼성 ‘헐값’ 유혹\n]', '[\n삼성전자 전자영수증 전세계 서비스센터 확대\n]', '[\n삼성전자, 갑자기 반도체 임원 20여 명 물갈이…왜?\n]', "[\n네이버클라우드-삼성전자, 국내 1호 '5G특화망' 상용화 협력\n]", '[\n[단독]삼성전

In [ ]:
titles[0].replace("[",'').replace("]",'').replace("\n",'')

'이찬희 삼성 준법위원장 “이재용 사면 결단내려야”'

In [ ]:
titles = list(map(lambda x: x.replace("[",'').replace("]",'').replace("\n",''), titles))

In [ ]:
contents = list(map(lambda x: x.replace("[",'').replace("]",'').replace("\n",'').replace("\t",''), contents))

In [ ]:
news_df = pd.DataFrame({'title':titles,'link':naver_urls,'content':contents})

In [ ]:
news_df

,title,link,content
0,이찬희 삼성 준법위원장 “이재용 사면 결단내려야”,https://n.news.naver.com/mnews/article/081/000...,삼성 7개사 최고경영진 간담회 앞서“국민의 뜻에 따라야 한다고 생각” 질문에 답하는...
1,"이재승 삼성전자 사장 ""국내 판매 냉장고 3대 중 2대 비스포크""",https://n.news.naver.com/mnews/article/003/001...,기사내용 요약비스포크 홈 2022 글로벌 행사 앞두고 비전 공개2019년 출시 후 ...
2,"이찬희 삼성 준법위원장 “이재용 부회장 사면, 국민의 뜻”",https://n.news.naver.com/mnews/article/018/000...,삼성 준법위·최고경영진 간담회 참석“재판으로 경영 어려움…결단 내려야”간담회선 “솔...
3,대기업 고용인원 170만명 1년새 6만명↑…삼성전자 11만명 '최대',https://n.news.naver.com/mnews/article/421/000...,"쿠팡, 작년 고용인원 3만명 늘어…고용 최다 그룹은 '삼성' 26.6만명자산 5조원..."
4,"삼성전자, 반도체연구소장 등 임원인사 단행",https://n.news.naver.com/mnews/article/014/000...,삼성전자 평택캠퍼스 사진=뉴스1 파이낸셜뉴스 삼성전자가 2일 반도체 선행기술 연구개...
5,삼성전자에 무슨 일이…반도체 임원 20여명 교체,https://n.news.naver.com/mnews/article/015/000...,자체 개발한 AP 엑시노스 수율 관련 조직 쇄신 인사반도체연구소도 조직 개편사진=연...
6,"삼성전자, 브라질서 에어컨 전력량 관리 '스마트싱스 에너지' 출시",https://n.news.naver.com/mnews/article/079/000...,핵심요약스마트싱스 에너지 서비스는 한국을 비롯해 미국·영국·인도에서 서비스를 시작했...
7,"삼성스팩 이틀 연속 상한가, 증권가도 당황했다… 무슨 일?",https://n.news.naver.com/mnews/article/023/000...,왕개미연구소“당황스럽네요. 삼성전자 인수·합병(M&amp;A)과 삼성스팩(SPAC)...
8,반도체 부사장 10명 교체…급박하게 돌아가는 '삼성시계' 뒷북비즈,https://n.news.naver.com/mnews/article/011/000...,"이재용, 반도체 부사장 10여명 전격교체7일 네덜란드 출장···인텔과 ARM 인수까..."
9,"삼성전자, 1분기 스마트워치 시장 2위… '갤럭시워치4'가 점유율 늘려",https://n.news.naver.com/mnews/article/417/000...,삼성전자가 올해 1분기 전세계 스마트워치 시장에서 2위를 기록하며 자존심을 지켰다...


# 잡플래닛 기업 평점 수집하기(프로젝트)

### 단일 리뷰 크롤링

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os 
os.chdir("/content/drive/MyDrive/금융특화 계층별 AI역량강화 교안/실습1_5. 웹 크롤링과 금융 데이터")

In [ ]:
!pip install selenium
!apt-get update
!apt install chromium-chromedriver

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 983 kB 24.5 MB/s 
     |████████████████████████████████| 138 kB 55.3 MB/s 
     |████████████████████████████████| 359 kB 53.0 MB/s 
     |████████████████████████████████| 4.0 MB 46.8 MB/s 
     |████████████████████████████████| 55 kB 4.5 MB/s 
     |████████████████████████████████| 58 kB 5.9 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.9 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
Get:1 https://

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from tqdm import tqdm

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver', chrome_options=chrome_options)

In [ ]:
corp = "SK하이닉스"
url = "https://www.jobplanet.co.kr/companies/20561/reviews/%EC%97%90%EC%8A%A4%EC%BC%80%EC%9D%B4%ED%95%98%EC%9D%B4%EB%8B%89%EC%8A%A4"

In [ ]:
# url 들어가기
driver.get(url)
driver.implicitly_wait(10) # 페이지가 전부 로딩될때까지 최대 10초 기다리기

In [ ]:
# 팝업 창이 뜨면 끄기 
try:
    driver.find_element(By.CLASS_NAME, "btn_close_x_ty1").click() 
except:
    pass

In [ ]:
review_list = driver.find_elements(By.CLASS_NAME, "content_wrap")

In [ ]:
review = review_list[0]

In [ ]:
date = review.find_elements(By.CLASS_NAME, "txt1")[-1].get_attribute('innerHTML')
print(date)

In [ ]:
score_list = review.find_element(By.CLASS_NAME, "ctbody_lft") 

In [ ]:
star_score = score_list.find_element(By.CLASS_NAME, "star_score").get_attribute('style')
print(star_score)

In [ ]:
# 항목별 평점
bl_score_list = score_list.find_elements(By.CLASS_NAME, "bl_score")
welfare_salary = bl_score_list[0].get_attribute("style")
work_life_balance = bl_score_list[1].get_attribute("style")
culture = bl_score_list[2].get_attribute("style")
promotion_opp = bl_score_list[3].get_attribute("style")
management = bl_score_list[4].get_attribute("style")
print(welfare_salary, work_life_balance, culture, promotion_opp, management)

In [ ]:
def convert_width_to_score(width):
    width_percent = int(width.split(' ')[1][:-2])
    score = int(width_percent / 20)
    return score

In [ ]:
print(convert_width_to_score(star_score))

In [ ]:
def extract_score_from_review(review):
    # 출력 초기화
    output = []
    # 리뷰에서 평점과 관련된 부분만 가져오기
    score_list = review.find_element(By.CLASS_NAME, "ctbody_lft")
    # 총 평점 가져와서 추가
    star_score = score_list.find_element(By.CLASS_NAME, "star_score").get_attribute('style')
    output.append(convert_width_to_score(star_score))
    # 항목별 평점 가져와서 추가
    other_score_elements = score_list.find_elements(By.CLASS_NAME, "bl_score")
    for element in other_score_elements:
        output.append(convert_width_to_score(element.get_attribute("style")))
    return output

In [ ]:
print(extract_score_from_review(review))

### 전체 리뷰 크롤링

#### 전체 페이지 수 획득

In [ ]:
btn_pglast = driver.find_element(By.CLASS_NAME, "btn_pglast")
num_pages = int(btn_pglast.get_attribute("href").split('=')[1])

#### 다음 페이지로 이동

In [ ]:
import time
def get_url(driver, url):
    while True:
        try:
            driver.get(url)
            driver.implicitly_wait(10) # 페이지가 전부 로딩될때까지 최대 10초 기다리기
            break
        except:
            driver.close()
            chrome_options = webdriver.ChromeOptions()
            chrome_options.add_argument('--headless')
            chrome_options.add_argument('--no-sandbox')
            chrome_options.add_argument('--disable-dev-shm-usage')
            driver = webdriver.Chrome('chromedriver', chrome_options=chrome_options)
            time.sleep(60)

#### 리뷰 수집
- 시간이 좀 걸립니다.

In [ ]:
import numpy as np
np.min([1,3])

In [ ]:
import pandas as pd
input_data = pd.read_excel("데이터/종목별_잡플래닛url.xlsx")
data = [] 

for corp, url in input_data[['종목명', 'URL']].iloc[:10].values:
    # 초기 url 들어가기
    get_url(driver, url)
    # 전체 페이지 수 계산
    btn_pglast = driver.find_element(By.CLASS_NAME, "btn_pglast")
    num_pages = int(btn_pglast.get_attribute("href").split('=')[1])
    # 페이지 순회 시작
    # for page in range(num_pages - 1):
    for page in range(np.min(30, num_pages - 1)):
        print("{}:{}/{}".format(corp, page+1, num_pages))
        current_url = url + "?page={}".format(page+1)
        get_url(driver, current_url)

        # 팝업 창이 뜨면 끄기 
        try:
            driver.find_element(By.CLASS_NAME, "btn_close_x_ty1").click()
            time.sleep(0.5)
        except:
            pass
        
        # 현재 페이지에서 데이터 가져오기
        while True:
            time.sleep(0.5)
            try:
                temp = []
                review_list = driver.find_elements(By.CLASS_NAME, "content_wrap")
                for review in review_list:
                    date = review.find_elements(By.CLASS_NAME, "txt1")[-1].get_attribute('innerHTML')
                    score = extract_score_from_review(review)
                    record = [corp, date] + score # score는 리스트임
                    temp.append(record)
                data += temp
                break
            except:
                driver.close()
                chrome_options = webdriver.ChromeOptions()
                chrome_options.add_argument('--headless')
                chrome_options.add_argument('--no-sandbox')
                chrome_options.add_argument('--disable-dev-shm-usage')
                driver = webdriver.Chrome('chromedriver', chrome_options=chrome_options)
                get_url(driver, current_url)
                time.sleep(60)

data = pd.DataFrame(data, columns = ["기업", "날짜", "전체평점", "승진기회및가능성", "복지및급여", "업무와삶의균형", "사내문화", "경영진"])

In [ ]:
display(data.head())

In [ ]:
# 데이터 정제
data = data.groupby(['기업', '날짜'], as_index = False)[["전체평점", "승진기회및가능성", "복지및급여", "업무와삶의균형", "사내문화", "경영진"]].mean()
data.sort_values(by = ['기업', '날짜'], inplace = True)
display(data.head(10))

In [ ]:
# data.to_csv("데이터/기업월별평점.csv", index = False, encoding = "euc-kr")

In [ ]:
data